In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

The question at hand: compute the steady-state "temperature" (value) in a square region whose boundary temperatures are known.
The Poisson eq. is written:

$$\frac{\partial^2{t(x,y)}}{\partial{x^2}}+\frac{\partial^2{t(x,y)}}{\partial{y^2}}=b(x,y)$$

By using the definition of derivatives, I can show that:

$$\frac{\partial^2{t(x,y)}}{\partial{x^2}}=\lim_{h\to0}\frac{t(x − h, y) − 2t(x, y) + t(x + h, y)}{h^2}$$
And same goes for $y$. That is why I added a $h^2={\delta_x}^2$ term in the reference solution.

In [ ]:
#-------------------------------------------------
#Define the boundary conditions (Yu Voon + Owen)
#-------------------------------------------------
L          = 1.0
ghost_zone = 1
N          = 3
dimension  = N**2

def xy(N):
    x = np.linspace( 0.0, L, N+2*ghost_zone ) 
    y = np.linspace( 0.0, L, N+2*ghost_zone )
    return x,y

# define a reference analytical solution
def u_ref(N):
    exwhy = xy(N)
    ref_func = np.zeros((N+2*ghost_zone,N+2*ghost_zone))
    for j in range(N+2*ghost_zone):
        for i in range(N+2*ghost_zone):
            ref_func[j,i] = np.sin(exwhy[0][i])*np.sin(exwhy[1][j])+1
    return ref_func

#boundary condition
def source_func(N):
    exwhy = xy(N)
    rho = np.zeros((N+2*ghost_zone,N+2*ghost_zone))
    for j in range(N+2*ghost_zone):
        for i in range(N+2*ghost_zone):
            rho[j,i] = (-2*np.sin(exwhy[0][i])*np.sin(exwhy[1][j]))*(L/(N+1))**2
    return rho

def initial_bound(N):
    if ghost_zone > 1:
        raise ValueError("Too big")
    ib = source_func(N)
    ur = u_ref(N)
    ib[ghost_zone,ghost_zone:-ghost_zone] -=  ur[0,ghost_zone:-ghost_zone] #top
    ib[-ghost_zone-1,ghost_zone:-ghost_zone] -=  ur[-1,ghost_zone:-ghost_zone]#bottom
    ib[ghost_zone:-ghost_zone,ghost_zone] -=  ur[ghost_zone:-ghost_zone,0]#left
    ib[ghost_zone:-ghost_zone,-ghost_zone-1] -=  ur[ghost_zone:-ghost_zone,-1] #right
    
    return ib[ghost_zone:-ghost_zone,ghost_zone:-ghost_zone]

In [ ]:
A         = -4.0*np.eye(dimension)

for idx_cell in range(dimension):
    idx_neighbor_list = []
    if (idx_cell%N  != 0 ):
        idx_neighbor_list.append(idx_cell-1) 
    if (idx_cell%N  != (N-1)):
        idx_neighbor_list.append(idx_cell+1)
    if (idx_cell//N != (N-1)):
        idx_neighbor_list.append(idx_cell+N)
    if (idx_cell//N != 0    ):
        idx_neighbor_list.append(idx_cell-N)
    if len(idx_neighbor_list) != 0:
        A[idx_cell,idx_neighbor_list ] = 1.0

b = initial_bound(N).ravel()

print(A)
# print(b)
# print(u_ref(N))
# print(source_func(N))
# print(initial_bound(N))

The problem lies with A. By printing out A (which is very sparse, with very many zeroes), we bottleneck ourselves with the size of A. For reference, N = 256 needs 32 GB of RAM without parallelization. We may need to simplify this by not calculating A, but instead using the equation:

$$−t(i − k) − t(i − 1) + 4t(i) − t(i + 1) − t(i + k) = 0$$

Which can give us the definition A: what A does, essentially; and use that to interact with d, making our code much faster (I think it is currently $O(N^4)$?) I think this has to be done before parallelization. Also note that the main diagonal is -4 due to rearrangement.

In [ ]:
#---------------------------
#CG Solver
#---------------------------
import time
x = np.zeros((N**2))  # first guess is zero vector
r = b  # r = b - A*x starts equal to b
d = r  # first search direction is r
start_time = time.time()
while (np.dot(r.T, r)/N**4 > 10e-16) == True:  # still iterating
    alpha = np.dot(r.T, r) / np.dot(d.T, np.dot(A, d))
    x = x + alpha * d  # step to next guess
    rnew = r - alpha * np.dot(A, d)  # update residual r
    beta = np.dot(rnew.T, rnew) / np.dot(r.T, r) #correction
    r = rnew
    d = r + beta * d  # compute new search direction
end_time = time.time()
print(end_time - start_time,' seconds')

In [ ]:
fig     = plt.figure(figsize=(9,3), dpi=320)
ref_sol = u_ref(N)[ghost_zone:-ghost_zone, ghost_zone:-ghost_zone]

ax1     = plt.subplot(131)
divider = make_axes_locatable(ax1)
cax1    = divider.append_axes('right', size='5%', pad=0.05)
im1     = ax1.imshow(x.reshape(N,N))
fig.colorbar(im1, cax=cax1, orientation='vertical')
ax1.axis("off")
ax1.set_title("CG Numerical")

ax2     = plt.subplot(132)
divider = make_axes_locatable(ax2)
cax2    = divider.append_axes('right', size='5%', pad=0.05)
im2     = ax2.imshow(ref_sol)
fig.colorbar(im2, cax=cax2, orientation='vertical')
ax2.axis("off")
ax2.set_title("Reference Solution")


ax3     = plt.subplot(133)
divider = make_axes_locatable(ax3)
cax3    = divider.append_axes('right', size='5%', pad=0.05)
im3     = ax3.imshow(x.reshape(N,N)-ref_sol)
fig.colorbar(im3, cax=cax3, orientation='vertical')
ax3.axis("off")
ax3.set_title("Difference")